<a href="https://colab.research.google.com/github/williamlidberg/Analyses-of-Environmental-Data-2/blob/main/modules/module_6/Assignment_6_geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geopandas
You can think of geopandas as pandas for vector data. By vector data I mean points, lines, polygons where each feature has a corresponding row in an attribute table. This is also refered to as tabular data. You have used geopandas in previous modules but the aim of this module is to prepare you for next module which is machine learning on vector data.

Start by installing geopandas and a few other packages

In [ ]:
!pip install geopandas
!pip install matplotlib # for plotting
!pip install folium # for data exploration
!pip install mapclassify # for data exploration

# Reading and writing data
The best thing about geopandas is that i can handle multiple file types such as GeoPackage, GeoJSON, Shapefile. Shapefiles can only contain about 4 GB of data and since many modern datasets is much larger than that the industry is slowly moving over to GeoPackage. You can easily read all these datatypes with the geopandas command "read_file()"

Start by downloading some data to work with

### Download ditches

In [ ]:
from urllib.request import urlretrieve
url = ('https://geodata.naturvardsverket.se/nedladdning/diken/Diken_Sverige/Diken_lansvis/Diken_K.zip')
filename = '/content/Diken_K.zip' # you need to adjust this path on your own computer if you are using anaconda.
urlretrieve(url, filename)
!unzip -o /content/Diken_K.zip -d /content/ditches

### Download wetlands

In [ ]:
from urllib.request import urlretrieve
url = ('https://geodata.naturvardsverket.se/nedladdning/VMI/ursprunglig_digitalisering/K_Blekinge_VMI.zip')
filename = '/content/K_Blekinge_VMI.zip' # you need to adjust this path on your own computer if you are using anaconda.
urlretrieve(url, filename)
!unzip -o '/content/K_Blekinge_VMI.zip' -d /content/wetlands

### Download protected areas

In [ ]:
from urllib.request import urlretrieve
url = ('https://geodata.naturvardsverket.se/nedladdning/naturvardsregistret/biosfarsomraden.zip')
filename = '/content/biosfarsomraden.zip' # you need to adjust this path on your own computer if you are using anaconda.
urlretrieve(url, filename)
!unzip -o '/content/biosfarsomraden.zip' -d /content/protected

### Read all data into geodataframes
All file types can be read with the same command.

In [ ]:
import geopandas as gpd
ditches = gpd.read_file('/content/ditches/Diken_K.gpkg') # Note that this is a geopackage
wetlands = gpd.read_file('/content/wetlands/K_Blekinge_VMI_Ytor.shp')
protected = gpd.read_file('/content/protected/BIOSFARSOMRADEN.shp')

You can also write the geodataframes to different file types. For example to save the wetlands polygons to a geojson we can specify the driver as GeoJson.

In [ ]:
wetlands.to_file("/content/wetlands.geojson", driver="GeoJSON")

## Task 1
Save the protected areas dataframe as a geopackage.

# Visalizing data
Geopandas includes a function to quickly inspect the data if the dataset is not too big. This works for the wetland polygons but it would probably crash google colab if you did this with the ditch dataframe. This is a very neat command to remember.



In [ ]:
wetlands.explore(column="NVKLASS") # move over the polygons to see the attibutes.

# Selecting data by attributes
It's quite common to be interested in parts of the dataset during an analysis. In most GIS programs you can run a tool named something along the lines "select by attributes". Since geopandas builds on pandas this can easily be achived with Python as well.

Lets pretend that we want to investigate ditches but only forest ditches. We can then select or slice the dataframe by that attribute. In our case: Typ

In [ ]:
ditches

In [ ]:
forest_ditches = ditches[ditches["Typ"] == "Skogsdike"] # Skogsdike is Swedish for forest ditch

Sometimes we want to make multiple selections at the same time. This is where the & sign can be used. Let's select forest ditches that are longer than 500 meters by adding the lenght attribute to the selection.

In [ ]:
forest_ditches_500m = ditches[(ditches["Typ"] == "Skogsdike") & (ditches["Langd_m"] > 500)] # Langd, or Längd is Swedish for length
forest_ditches_500m.explore()

### Task 2
Make a selection on the wetlands geodataframe and awsner the question: How many wetlands have a high nature value (Högt naturvärde), an area over 10 hectares and a perimiter less than 2000 meters.

# Spatial relations
One of the most commonly used operations within GIS is to select features from one dataset that falls within the features of another dataset. This is commonly refered to as spatial join or intersect.

Lets select all wetlands that fall within the protective area using geopandas.

In [ ]:
selected_wetlands = gpd.sjoin(wetlands, protected, op='within')
selected_wetlands.explore(column="NVKLASS")

You can reverse this selection by using the ~ sign.

In [ ]:
non_intersecting_wetlands = wetlands[~wetlands.index.isin(selected_wetlands.index)]
non_intersecting_wetlands .explore(column="NVKLASS")

The spatial join operation have many different options:
*   contains
*   within
*   touches
*   crosses
*   overlaps
*   equals

## Task 3
Use geopandas to select ditches that crosses the wetlands and then show them on a map.

# Math operations
Another common operation is to summarize the area/lenght or features that falls within other features. For example we can summarize the lenght of ditches within the wetlands.

In [ ]:
joined_gdf = gpd.sjoin(ditches, wetlands, how="inner", op='intersects') # intersect ditches with wetlands
joined_gdf['line_length'] = joined_gdf.geometry.length # calculate lenght of intersected ditches
summary = joined_gdf.groupby('NVKLASS')['line_length'].sum() #summarize the lenght of ditch lines within each wetland (right refers to the right geodataframe in the intersect)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(x=summary.index, y=summary.values)
plt.xlabel('NVKLASS')
plt.ylabel('Length of Ditch Lines in meter')
plt.title('Length of Ditch Lines within Wetlands by NVKLASS')
plt.xticks(rotation=90)
plt.show()


This does not tell us much since we dont take the size of each wetland into account. Lets extract the area of each wetland and devide the ditch lenght by wetland area.

In [ ]:
import pandas as pd

wetlands['wetland_area'] = wetlands.geometry.area
joined_gdf = gpd.sjoin(ditches, wetlands, how="inner", op='intersects')
joined_gdf['line_length'] = joined_gdf.geometry.length
joined_gdf['length_per_unit_area'] = joined_gdf['line_length'] / joined_gdf['wetland_area']

summary = joined_gdf.groupby('NVKLASS')['length_per_unit_area'].sum()
summary = summary.reset_index()

In [ ]:
sns.barplot(x='NVKLASS', y='length_per_unit_area', data=summary)
plt.xticks(rotation=90)
plt.show()

# Buffers
Creating a buffer around features is also a very common tool for geospatial analysis.

In [ ]:
import geopandas as gpd

wetlands_buffer = wetlands.geometry.buffer(500) # buffer wetlands with 500 meters
wetlands_buffer_gdf = gpd.GeoDataFrame(geometry=wetlands_buffer) # create a new geodataframe from the buffered geometries

wetlands_buffer_gdf.explore()

If you look at the map you can see that some polygons are overlapping each other. Sometimes this is not what you want and in those cases they can be merged by dissolving the overlap.

In [ ]:
import geopandas as gpd

wetlands_buffer = wetlands.buffer(500)
singlepart = wetlands_buffer.unary_union # Dissolve overlapping buffers
singlepart = gpd.GeoDataFrame(geometry=[singlepart])
singlepart.crs = wetlands.crs # set crs to the new data
singlepart.explore()

## Task 4
Lets say we want to restore some drained wetlands but at the same time we dont want to destroy some cultural sites nearby. We can help with this by finding drained wetlands that do not have any cultural sites within 300 meters.  Download water-related cultural sites from [Länsstyrelsen](https://ext-geodatakatalog.lansstyrelsen.se/GeodataKatalogen/srv/swe/catalog.search#/search?isTemplate=n&referenceDateRange=%7B%22range%22:%7B%22referenceDateRange%22:%7B%22gte%22:null,%22lte%22:null,%22relation%22:%22within%22%7D%7D%7D&metadataDateRange=%7B%22range%22:%7B%22metadataDateRange%22:%7B%22gte%22:null,%22lte%22:null,%22relation%22:%22within%22%7D%7D%7D&metadataAndReferenceDateRange=%7B%22range%22:%7B%22metadataAndReferenceDateRange%22:%7B%22gte%22:null,%22lte%22:null,%22relation%22:%22within%22%7D%7D%7D&sortBy=relevance&sortOrder=&query_string=%7B%22spatialRepresentationType%22:%7B%22vector-point%22:true%7D,%22ownerOrgName%22:%7B%22L%C3%A4nsstyrelsen%20Blekinge%20l%C3%A4n%22:true%7D%7D&from=1&to=30) and select wetlands with some nature values (Vissa naturvärden) that are more than 300 meters away from cultural sites. Save the selected wetlands as a geopackage.

In [ ]:
# You can download the data using urllib
from urllib.request import urlretrieve
url = ('https://ext-dokument.lansstyrelsen.se/gemensamt/geodata/ShapeExport/Lstk.Inventerade_vattenanknutna_kulturmiljoer.zip')
filename = '/content/Lstk.Inventerade_vattenanknutna_kulturmiljoer.zip' # you need to adjust this path on your own computer if you are using anaconda.
urlretrieve(url, filename)
!unzip -o '/content/Lstk.Inventerade_vattenanknutna_kulturmiljoer.zip' -d /content/culture

# Data aggregation
There is a global geospatial index based around nested hexagon polygons which is called H3. It was developed by [Uber](https://www.uber.com/en-SE/blog/h3/) and later made into open source for everyone to use.

* H3 is a hierarchical geospatial index.
* H3 was developed to address the challenges of Uber's data science needs.
* H3 can be used to join disparate data sets.
* In addition to the benefits of the hexagonal grid shape, H3 includes features for modeling flow.
* H3 is well suited to apply machine learning to geospatial data.

H3 can be very useful if you are working in an area and need to aggreagate multiple data soruces. Since the polygons have uniqe IDs and are constant for everyone you can easily share data with other people based on the H3 index.

Also, [hexagons are the bestagons](https://www.youtube.com/watch?v=thOifuHs6eY)

In [ ]:
!pip install tobler
!pip install h3
!pip install h3pandas

First we need to select which geographical area we want to work with, start by downloading some Swedish regions from [SCB}(https://www.scb.se/hitta-statistik/regional-statistik-och-kartor/regionala-indelningar/digitala-granser/)


In [ ]:
from urllib.request import urlretrieve
url = ('https://www.scb.se/contentassets/3443fea3fa6640f7a57ea15d9a372d33/shape_svenska_240104.zip')
filename = '/content/Sweden_shapefile.zip' # you need to adjust this path on your own computer if you are using anaconda.
urlretrieve(url, filename)
!unzip -o /content/Sweden_shapefile.zip -d /content/sweden
!unzip -o /content/sweden/LanSweref99TM.zip -d /content/sweden/lan

We can keep working with Blekinge since its so small and we already have some data for it.

In [ ]:
sweden = gpd.read_file('/content/sweden/lan/Lan_Sweref99TM_region.shp')
blekinge = sweden[sweden["LnNamn"] == "Blekinge"] # Skogsdike is Swedish for forest ditch
blekinge.plot()

With this polygon we can now call on the H3 api and create a hexagon index for Blekine. There are multiple resoltuions where the bigest hexagon is 4,357,449.41 km² and the smallest 0.895 m²

The resolution parameter goes from 0 to 15 but due to limited RAM on colab we will use 7 here.

In [ ]:
import h3pandas
import tobler
blekinge_hexagons = tobler.util.h3fy(blekinge, resolution=7, clip=False, buffer=False, return_geoms=True)
blekinge_hexagons.explore()

Note the hex_id column. Different resolutions have different IDs but each polygon ID is uniqe which means that you can use it to join and share data based on this global ID.

In [ ]:
blekinge_hexagons

Lets summarize the number of ditch lines within each hexagon.

In [ ]:
import geopandas as gpd

# make sure that the coordinate system is the same
ditches  = ditches.to_crs(blekinge_hexagons.crs)

# Perform a spatial join
joined = gpd.sjoin(blekinge_hexagons, ditches, op='intersects', how='left')

# Group by hex_id and count the number of sites within each hexagon
summary = joined.groupby('hex_id').size().reset_index(name='ditch_count')

# Merge the summary with the hexagon GeoDataFrame to add the cnumber of sites within each hexagon
hexagons_with_sites = blekinge_hexagons.merge(summary, on='hex_id', how='left')
hexagons_with_sites.explore('ditch_count')

However, since some ditches are longer than others it makes more sense to summarize the length of ditches instead of the number of line segments.

In [ ]:
import geopandas as gpd

ditches = ditches.to_crs(blekinge_hexagons.crs)
joined = gpd.sjoin(blekinge_hexagons, ditches, op='intersects', how='left')

# Calculate the length of each ditch line
joined['line_length'] = joined.geometry.length

# Group by hex_id and sum the length of ditch lines within each hexagon
summary = joined.groupby('hex_id')['line_length'].sum().reset_index()
hexagons_with_ditch_length = blekinge_hexagons.merge(summary, on='hex_id', how='left')
hexagons_with_ditch_length.explore('line_length')

Finally we aggregate more data to the hexagons. For example, lets also add the area of wetlands within each hexagon.

In [ ]:

wetlands = wetlands.to_crs(hexagons_with_ditch_length.crs)
joined_wetlands = gpd.sjoin(hexagons_with_ditch_length, wetlands, op='intersects', how='left')

# Calculate the area of each wetland polygon
joined_wetlands['wetland_area'] = joined_wetlands.geometry.area

# Group by hex_id and sum the area of wetlands within each hexagon
summary_wetlands = joined_wetlands.groupby('hex_id')['wetland_area'].sum().reset_index()

hexagons_with_both_ditch_lenght_and_wetland_area = hexagons_with_ditch_length.merge(summary_wetlands, on='hex_id', how='left')
hexagons_with_both_ditch_lenght_and_wetland_area

### Task 5
Add the number of cultural sites to these hexagons so you get ditch length, wetlands area and number of cultural sites within the same geodataframe.

# Conclusions
Geopandas is a powerful tool to wrangle geospatial vector data and can read and write multiple different file formats. Aggrigating data is very useful for further analyses and we will use this skill in the next module to combine aggregated vector data with machine learning and implement the model to predict things on the H3 geospatial index. Here is an example of how it can be used in the real world: https://beta.behovskartan.se/